In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization, MaxPooling2D, Dropout, Input
from keras import metrics
from keras.models import model_from_json
from keras.callbacks import Callback
from keras import optimizers
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score

Using TensorFlow backend.


In [2]:
ip_folder = "./input_x"

X_train = np.load(ip_folder + "/X_train.npy")
y1_train = np.load(ip_folder + "/y1_train.npy")
y2_train = np.load(ip_folder + "/y2_train.npy")

X_test = np.load(ip_folder + "/X_test.npy")
y1_test = np.load(ip_folder + "/y1_test.npy")
y2_test = np.load(ip_folder + "/y2_test.npy")


print (X_train.shape, X_test.shape, y1_train.shape, y1_test.shape)

((1083, 480, 640, 3), (541, 480, 640, 3), (1083, 3), (541, 3))


In [4]:
def getBranch(inp, out_neuron, out_act, name):
    x = Dense(512, activation="relu")(inp)
    x = Dropout(0.4)(x)
    x = BatchNormalization()(x)
    x = Dense(out_neuron, activation=out_act, name=name)(x)
    return x

In [5]:
inp = Input(shape=(480,640,3))
cnn = Conv2D(32, kernel_size=3, strides=1, activation="relu")(inp)
cnn = BatchNormalization()(cnn)
cnn = MaxPooling2D(pool_size=2, strides=1)(cnn)
cnn = Conv2D(8, kernel_size=1, strides=1, activation="relu")(cnn)
cnn = BatchNormalization()(cnn)
cnn = Dropout(rate=0.4)(cnn)
cnn = Flatten()(cnn)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
classification_head = getBranch(cnn, 3, "softmax", "class_head")
regression_head = getBranch(cnn, 4, "softmax", "reg_head")
model = Model(inputs=inp, outputs=[classification_head, regression_head])
model.output_shape

[(None, 3), (None, 4)]

In [7]:
model.compile(optimizer="adam",
              loss={"class_head": "categorical_crossentropy", "reg_head": "mean_squared_error"},
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, [y1_train, y2_train],
                    validation_data=(X_test, [y1_test, y2_test]),
                    epochs=2, batch_size=10)

Instructions for updating:
Use tf.cast instead.
Train on 1083 samples, validate on 541 samples
Epoch 1/2


In [ ]:
# Final evaluation of the model
y1_pred, y2_pred = model.predict(X_test)


In [ ]:
y1_test = y1_test >= 0.5
y2_test = y2_test >= 0.5

y1_pred = y1_pred >= 0.5
y2_pred = y2_pred >= 0.5

cm_y1 = confusion_matrix(y1_test.argmax(axis=1), y1_pred.argmax(axis=1))
cm_y2 = confusion_matrix(y2_test.argmax(axis=1), y2_pred.argmax(axis=1))

print(cm_y1)
print(cm_y2)